In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import *
from lightgbm import LGBMClassifier
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pickle

ModuleNotFoundError: No module named 'seaborn'

In [2]:
train = pd.read_csv(r'train.csv')
test = pd.read_csv(r'test.csv')
sub = pd.read_csv(r'test.csv')

In [3]:
train.head(10)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0
5,ACUTYTWS,Male,56,RG261,Self_Employed,X1,32,No,544163,Yes,0
6,ETQCZFEJ,Male,62,RG282,Other,X3,20,NaN,1056750,Yes,1
7,JJNJUQMQ,Female,48,RG265,Self_Employed,X3,13,No,444724,Yes,0
8,ZMQFYKCB,Female,40,RG283,Self_Employed,X2,38,No,1274284,No,0
9,NVKTFBA2,Female,55,RG268,Self_Employed,X2,49,Yes,2014239,No,0


In [4]:
train.columns

Index(['ID', 'Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code',
       'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active',
       'Is_Lead'],
      dtype='object')

In [5]:
test.dtypes

ID                     object
Gender                 object
Age                     int64
Region_Code            object
Occupation             object
Channel_Code           object
Vintage                 int64
Credit_Product         object
Avg_Account_Balance     int64
Is_Active              object
dtype: object

In [6]:
train.Is_Active.value_counts()

No     150290
Yes     95435
Name: Is_Active, dtype: int64

In [7]:
train.fillna("NA",inplace=True)
test.fillna("NA",inplace=True)

In [8]:
gender_mapper = {"Female":1,"Male":0}
train['Gender'] = train.Gender.map(gender_mapper)
test['Gender'] = test.Gender.map(gender_mapper)

In [9]:
occ_mapper = {"Self_Employed":1,"Salaried":0,"Other":2,"Entrepreneur":3}
train['Occupation'] = train.Occupation.map(occ_mapper)
test['Occupation'] = test.Occupation.map(occ_mapper)

In [10]:
train['Region_Code'] = train.Region_Code.str[2:].astype(int)
test['Region_Code'] = test.Region_Code.str[2:].astype(int)

In [11]:
train['Channel_Code'] = train.Channel_Code.str[1:].astype(int)
test['Channel_Code'] = test.Channel_Code.str[1:].astype(int)

In [12]:
cp_mapper = {"No":1,"NA":0,"Yes":2}
train['Credit_Product'] = train.Credit_Product.map(cp_mapper)
test['Credit_Product'] = test.Credit_Product.map(cp_mapper)

In [13]:
is_active_mapper = {"Yes":1,"No":0}
train['Is_Active'] = train.Is_Active.map(is_active_mapper)
test['Is_Active'] = test.Is_Active.map(is_active_mapper)

In [14]:
train['age_bin'] = np.where((train.Age>40)&(train.Age<65),1,0)
test['age_bin'] = np.where((test.Age>40)&(test.Age<65),1,0)

In [15]:
selected_columns = ['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code',
       'Vintage', 'Credit_Product', 'Avg_Account_Balance', 'Is_Active']
len(selected_columns)

9

In [16]:
X = train[selected_columns]
y = train.Is_Lead

In [17]:
X.head(20)

,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
0,1,73,268,2,3,43,1,1045696,0
1,1,30,277,0,1,32,1,581988,0
2,1,56,268,1,3,26,1,1484315,1
3,0,34,270,0,1,19,1,470454,0
4,1,30,282,0,1,33,1,886787,0
5,0,56,261,1,1,32,1,544163,1
6,0,62,282,2,3,20,0,1056750,1
7,1,48,265,1,3,13,1,444724,1
8,1,40,283,1,2,38,1,1274284,0
9,1,55,268,1,2,49,2,2014239,0


In [18]:
y.head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     1
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    1
16    1
17    1
18    0
19    0
Name: Is_Lead, dtype: int64

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42,stratify = y)

In [20]:
model =LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=30,
    max_depth=7,
    learning_rate=0.1,
    n_estimators=150,
    subsample_for_bin=200000,
    objective=None,
    class_weight=None,
    min_split_gain=0.0,
    min_child_weight=0.001,
    min_child_samples=20,
    subsample=1.0,
    subsample_freq=0,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.0,
    random_state=42,
    n_jobs=-1,
    silent=True,
    importance_type='split'
)

In [21]:
model.fit(X_train,y_train)

LGBMClassifier(max_depth=7, n_estimators=150, num_leaves=30, random_state=42)

In [22]:
model.predict(X_test[selected_columns])

array([0, 0, 0, ..., 1, 1, 0], dtype=int64)

In [23]:
model.fit(X,y)

LGBMClassifier(max_depth=7, n_estimators=150, num_leaves=30, random_state=42)

In [24]:
filename = 'lgbm_model.plk'

In [25]:
pickle.dump(model, open(filename, 'wb'))